# Import

In [10]:
import os
import requests
import mlflow
import pandas as pd
from sklearn.metrics import accuracy_score

# MLFlow Tracking Server Setting

In [14]:
MLFLOW_SERVER_URI = 'http://127.0.0.1:5000'

In [15]:
mlflow.set_tracking_uri(MLFLOW_SERVER_URI)

# Load Dataset

In [5]:
dataset = pd.read_csv(os.path.join(os.getcwd(), 'data', 'dataset_processed.csv'))

target = dataset['Survived']
dataset = dataset.drop(['Survived'], axis=1)

dataset.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S
0,0.271174,0.125,0.0,0.014151,0,0,1,0,1,0,0,0,1
1,0.472229,0.125,0.0,0.139136,1,0,0,1,0,1,0,0,0
2,0.321438,0.000,0.0,0.015469,0,0,1,1,0,0,0,0,1
3,0.434531,0.125,0.0,0.103644,1,0,0,1,0,0,0,0,1
4,0.434531,0.000,0.0,0.015713,0,0,1,0,1,0,0,0,1


# Load/Inference Staging Model

In [6]:
staging_model_registry = "staging.titanic.random_forest_clf"
staging_version = "2"
staging_model_uri = f"models:/{staging_model_registry}/{staging_version}"

staging_model = mlflow.pyfunc.load_model(staging_model_uri)

In [7]:
prediction = staging_model.predict(dataset)

print(
    f"Accuracy of Staging Model : {accuracy_score(target, prediction)}"
)

Accuracy of Staging Model : 0.9214365881032548


# Load/Inference Production Model

In [11]:
prod_model_registry = "prod.titanic.random_forest_clf"
prod_version = "1"
prod_model_uri = f"models:/{prod_model_registry}/{prod_version}"

prod_model = mlflow.pyfunc.load_model(prod_model_uri)

In [12]:
prediction = prod_model.predict(dataset)

print(
    f"Accuracy of Production Model : {accuracy_score(target, prediction)}"
)

Accuracy of Production Model : 0.9225589225589226


# Model Serving Using MLFlow

1. terminal > `export MLFLOW_TRACKING_URI=http://127.0.0.1:5000`
2. terminal > `mlflow models server -m models:/prod.titanic.random_forest_clf -p 5001 --no-conda`

In [ ]:
host = "127.0.0.1"
port = "5001"
url = f"http://{host}:{port}/invocations"

json_data = {
    "dataframe_split" : dataset[:10].to_dict(orient="split")
}

response = requests.post(url, json=json_data)

print(
    f"Pyfunc 'predict_interval' : \n${response.json()}"
)

Pyfunc 'predict_interval' : 
${'predictions': [0, 1, 1, 1, 0, 0, 0, 0, 1, 1]}


In [19]:
json_data = {
    "dataframe_split" : dataset[20:40].to_dict(orient="split")
}

response = requests.post(url, json=json_data)

print(
    f"Pyfunc 'predict_interval' : \n${response.json()}"
)

Pyfunc 'predict_interval' : 
${'predictions': [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]}


# Deploy on Cloud

## AWS SageMaker

- `mlflow sagemaker build-and-push-container`
- `mlflow sagemaker deploy {parameters}`

## AzureML

- `mlflow azureml export -m {model path} -o test-output`